In [1]:
import numpy as np

DIMENSIONS = (36, 36)

## Turn asteroid belt into 2D numpy array in which 1 represents an asteroid

In [2]:
with open("./input.txt") as f:
    asteroid_belt = f.read().replace("\n", "")

asteroid_belt = asteroid_belt.replace(".", "0").replace("#", "1")
asteroid_belt = np.array(list(map(int, list(asteroid_belt))))
asteroid_belt.shape = (36, 36)
asteroid_belt = asteroid_belt.transpose()

## Store (x, y) of asteroid locations in an array

In [3]:
asteroid_locs = [
    (x, y) for x in range(36)
    for y in range(36)
    if asteroid_belt[x, y] == 1
]

In [4]:
from typing import Tuple
from fractions import Fraction

def check_point_on_line(
    point1: Tuple[int, int], point2: Tuple[int, int], slope: Fraction
) -> bool:
    """Checkes whether point2 and point1 lie on line created by slope
    
    Pass None for slope if it is a vertical line
    """
    if slope is None:
        return point1[0] == point2[0]

    y2_minus_y1 = point2[1] - point1[1]
    x2_minus_x1 = point2[0] - point1[0]
    slope_between_points = Fraction(y2_minus_y1, x2_minus_x1)

    if slope == 0 and slope_between_points == 0:
        return True
    elif slope_between_points == 0:
        return False    

    return (slope / slope_between_points) == 1


assert check_point_on_line((0, 0), (1, 2), Fraction(2))
assert check_point_on_line((0, 0), (-1, -2), Fraction(2))
assert check_point_on_line((2, 2), (4, 2), Fraction(0))
assert check_point_on_line((2, 2), (2, 4), None)

In [5]:
def get_slope_between_points(
    point1: Tuple[int, int], point2: Tuple[int, int]
) -> Fraction:
    """Given two points, returns Fraction object representing slope

    Returns None for vertical slope
    """
    if point1[0] == point2[0]:
        return None

    return Fraction(point2[1]-point1[1], point2[0]-point1[0])


assert get_slope_between_points((0, 0), (1, 2)) == Fraction(2, 1)
assert get_slope_between_points((2, 0), (2, 2)) == None

## Determine how many asteroids are reachable from a given asteroid

For each asteroid, iterate over all over asteroids and compute the slope between the two asteroids.

Each asteroid can have, at most, two other reachable asteroids along the same line. One asteroid at a position "less than" the current asteroid's location or a position "greater than" the asteroid's location.

Consider an asteroid located at (1, 1) and other asteroids located at (0, 0), (2, 2), and (3, 3). There are two reachable asteroids, one at (0, 0) and one at (2, 2). The one at (3, 3) is unreachable because it's blocked by (2, 2). These all lie along the same line but relative to (1, 1) there is one to the left and one to the right.

To count the  number of reachable asteroids from a given location, we create a dictionary of slopes to other asteroids and whether an asteroid is at a position greater or less than the current location. The number of entries in the dictionary then gives the number of reachable asteroids.

In [6]:
reachable_asteroids = np.zeros((36, 36))

for asteroid_loc in asteroid_locs:
    # get list of all asteroids other than current one
    asteroids_to_check = [asteroid for asteroid in asteroid_locs if asteroid != asteroid_loc]

    # map a slope to an asteroid that reaches it along given slope
    slope_to_asteroid = {}

    for asteroid in asteroids_to_check:
        slope = get_slope_between_points(asteroid_loc, asteroid)

        if slope not in slope_to_asteroid:
            slope_to_asteroid[slope] = {}

        if slope is None:
            if asteroid[1] > asteroid_loc[1]:
                slope_to_asteroid[slope]["Greater"] = asteroid
            else:
                slope_to_asteroid[slope]["Less"] = asteroid
        else:
            if asteroid[0] > asteroid_loc[0]:
                slope_to_asteroid[slope]["Greater"] = asteroid
            else:
                slope_to_asteroid[slope]["Less"] = asteroid

    # now, check how many asteroids we can reach
    num_reachable_asteroids = sum(len(reachable) for reachable in slope_to_asteroid.values())
    reachable_asteroids[asteroid_loc] = num_reachable_asteroids

In [7]:
reachable_asteroids.max()

276.0